Created on May 01, 2020

@author:     Tim Schmittmann

Description: Nuclei segmentation using Mask-RCNN based on https://github.com/ruchikaverma-iitg/Nuclei-Segmentation/blob/master/Nuclei_Segmentation_testing_code.ipynb
            


In [0]:
# For use in colab
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/gdrive/')

In [0]:
# For use in colab. Adjust path depending on your setup 
%cd /content/gdrive/My\ Drive/informatik/privat/AML\ Git/dl-based-image-cell-segmentation-with-mask-rcnn/

!pwd

In [0]:
%tensorflow_version 1.x

In [0]:
from PIL import Image
import skimage
import imageio
import os,sys
import glob
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
#from PIL import Image
import scipy
import scipy.ndimage
from pathlib import Path
from skimage.measure import find_contours
from matplotlib.patches import Polygon

# Import Mask RCNN and load my weights of my trained model. MRCNN is based on https://github.com/matterport/Mask_RCNN
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log
import nucleus
import tensorflow as tf

In [0]:
# For cross-compatibility use pipreqs to save only required dependencies
!pip install pipreqs

In [0]:
!pipreqs --force

In [0]:
# In case pipreqs missed something
!pip freeze > pipfreeze.requirements.txt

In [0]:
# Dataset directory
ROOT_DIR = os.path.abspath("")
WEIGHTS_DIR = os.path.join(ROOT_DIR, "weights/")
DATASET_DIR = os.path.join(ROOT_DIR, "images/")
RESULTS_DIR = os.path.join(ROOT_DIR, "results/")

In [0]:
if not os.path.isdir(DATASET_DIR):
  os.mkdir(DATASET_DIR)
if not os.path.isdir(WEIGHTS_DIR):
  os.mkdir(WEIGHTS_DIR)
if not os.path.isdir(RESULTS_DIR):
  os.mkdir(RESULTS_DIR)

In [0]:
"""Download weights file. Original: https://drive.google.com/drive/folders/16oPaebQnZCMzEsEGvhSVPMvEhbKJPATQ
   And move it to "weights/mask_rcnn_weights.h5"
   Sometimes the automatic download+unzip routine fails in colab and results in corrupt files. 
   In that case simply download the originals and move the files into the correct directory.
"""
!wget -O weights.zip https://schmittmann.me/index.php/s/2DubYqV9pzs4ax8/download
!unzip weights.zip -d weights

In [0]:
"""Download and extract example dataset. 
   Provided by Dr. Middeke https://cloudstore.zih.tu-dresden.de/index.php/s/FQ2bjHaj6tR2bmg/download
   Images are sorted into directories by their (estimated) magnification.
"""
!wget -O images.zip https://schmittmann.me/index.php/s/or393w0KtB8EQjV/download 
!unzip images.zip -d images

In [0]:
# Initialize Mask RCNN model with pretrained weights
config = nucleus.NucleusInferenceConfig()
config.display()
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0
with tf.device(DEVICE):
    nmodel = modellib.MaskRCNN(mode="inference",
                              model_dir=os.getcwd(),
                              config=config)
weights_path = os.path.join(WEIGHTS_DIR, "mask_rcnn_weights.h5")
nmodel.load_weights(weights_path, by_name=True)  

In [0]:
def img_to_tiles(img, target_tile_size):
  """Split image in evenly sized tiles. Crops any leftover pixel that are not large enough for another tile"""
  S = target_tile_size
  return [img[x:x+S,y:y+S] for x in range(0,int(img.shape[0] / S) * S,S) for y in range(0,int(img.shape[1] / S) * S,S)]

In [0]:
def prepare_image(img, scale_factor):
  """Simple image preparation for Mask RCNN model"""
  img = skimage.exposure.equalize_adapthist(img)
  img = skimage.exposure.adjust_log(img)
  img = skimage.transform.rescale(img, (scale_factor, scale_factor, 1))
  img = img * 255.0
  return img

In [0]:
def make_results_type_magnification_dir(result_type, magnification):
  """Helper function to create result directories"""
  results_type_dir = os.path.join(RESULTS_DIR, result_type)
  if not os.path.isdir(results_type_dir):
    os.mkdir(results_type_dir)
  results_type_magnification_dir = os.path.join(results_type_dir, str(int(magnification)))
  if not os.path.isdir(results_type_magnification_dir):
    os.mkdir(results_type_magnification_dir)
  return results_type_magnification_dir

In [0]:
def save_figure_results(results_type, magnification, filename):
  """Helper function to save figures and create necessary directories"""
  results_dir = make_results_type_magnification_dir(results_type, magnification)
  plt.savefig(os.path.join(results_dir, filename+'.png'))

In [0]:
def save_segmented_cells(filename, magnification, results_type, segmented_cells):
  """Loop all segmented and extracted cell patches and save them inside 
  their specific results_type+magnification directory"""
  for i in range(len(segmented_cells)):
    cell = segmented_cells[i]
    results_dir = make_results_type_magnification_dir(results_type, magnification)
    
    plt.imsave(os.path.join(results_dir, filename+'_cell'+str(i)+'.png'), cell)


In [0]:
def save_cell_masks(filename, magnification, results_type_dir, original_tile, cell_masks):
  """Loop all cell masks and extract the corresponding patches from the original tile. 
  Save the patches inside the specific results_type+magnification directory"""
  cells = []
  for i in range(cell_masks.shape[2]):
    cell_mask = np.copy(original_tile)
    cell_mask[cell_masks[:,:,i] == 0] = 0
    cells.append(crop_image_only_outside(cell_mask))
  save_segmented_cells(filename, magnification, results_type_dir, cells)

In [0]:
def rois_to_masks(rois, tile):
  """Transforms boundary boxes in format [xmin, ymin, xmax, ymax] into binary masks with the shape of tile"""
    roi_masks = []
    for roi in rois:
      roi = roi.astype(int)
      roi_mask = np.zeros((tile.shape[0], tile.shape[1]))
      roi_mask[roi[0]:roi[2], roi[1]:roi[3]] = 1
      roi_masks.append(roi_mask)
    return roi_masks

In [0]:
def get_extended_cell_rois(results, original_tile, rescaled_rois):
  """Extend cell rois by 1/4th of their size"""
  for roi in rescaled_rois:
    dx = int((roi[2] - roi[0]) / 4)
    dy = int((roi[3] - roi[1]) / 4)
    roi[0] -= dx
    roi[1] -= dy
    roi[2] += dx
    roi[3] += dy
  return rescaled_rois

In [0]:
def unify_cell_roi_masks(tile, cell_roi_masks):
  """Simple union of cell_roi_masks"""
  cell_roi_masks_union = np.zeros(tile.shape)
  for roi_mask in cell_roi_masks:
    cell_roi_masks_union[roi_mask == 1] = 1
  return cell_roi_masks_union


In [0]:
def get_avg_foreground_color(tile, cell_roi_masks_union):
  """Calculate average color of cell pixels in tile.
  
  Parameters
  ----------
  tile : ndarray
      The image to calculate foreground color from
  cell_roi_masks_union : ndarray
      Binary mask to identify cell pixels
      
  Returns
  -------
  foreground_color : ndarray
      Average rgb foreground color 
  """
  foreground = np.copy(tile)
  foreground = foreground[cell_roi_masks_union == 1]

  foreground_color = np.average(foreground, axis=(0))
  return foreground_color

In [0]:
def get_avg_background_color(tile, cell_roi_masks_union):
  """Calculate artificial average background color of non-cell pixels in tile.
  To better separate background and foreground, we artificially increase the difference 
  between foreground and background.

  Parameters
  ----------
  tile : ndarray
      The image to calculate background color from
  cell_roi_masks_union : ndarray
      Binary mask to identify non-cell pixels
      
  Returns
  -------
  background_color : ndarray
      Enhanced average rgb background color 
  """
  background = np.copy(tile)
  background = background[cell_roi_masks_union == 0]
  background_color = np.average(background, axis=(0))

  diff = background_color-get_avg_foreground_color(tile, cell_roi_masks_union)
  # Move background further away from foreground
  background_color += diff
  background_color = np.clip(background_color, 0,255)
  return background_color

In [0]:
def crop_image_only_outside(img,tol=0):
    """Crop image to remove all pixels below threshold/tolerance on the borders
    of the image (https://codereview.stackexchange.com/questions/132914/crop-black-border-of-image-using-numpy)

    Parameters
    ----------
    img : ndarray
        Image to crop
    tol : int, optional
        Threshold. Crop any pixels on the borders of img below this. 

    Returns
    -------
    img : ndarray
        Cropped image
    """
    mask = img>tol
    if img.ndim==3:
        mask = mask.all(2)
    m,n = mask.shape
    mask0,mask1 = mask.any(0),mask.any(1)
    col_start,col_end = mask0.argmax(),n-mask0[::-1].argmax()
    row_start,row_end = mask1.argmax(),m-mask1[::-1].argmax()
    return img[row_start:row_end,col_start:col_end]

In [0]:
def get_segmented_cells_rois(tile, rescaled_masks, cell_roi_masks, cell_roi_masks_union, background_color):
  """Extract the segmented cells as rectangular extraction from the original tile based on their roi_masks.
  Also uses the rescaled_masks to remove neighbouring cells from the final extracted patches.

  Parameters
  ----------
  tile : ndarray
      The (original) tile to extract the rectangular cell images from
  rescaled_masks : ndarray
      Binary masks used to find neighbouring cells. Same shape as tile
  cell_roi_masks : ndarray
      The roi masks used to extract the rectangular cell images from tile
  cell_roi_masks_union : ndarray
      Union of all cell_roi_masks to quickly get all cell/non-cell pixels 
  background_color : ndarray
      The average background color of non-cell pixels of tile

  Returns
  -------
  cell_rois : ndarray
      Extracted the rectangular cell images from tile
  cell_rois_wo_neighbor : ndarray
      Extracted the rectangular cell images from tile with neighbouring cells replaced by background_color  
  """
  img_homogenous_background = np.copy(tile)
  img_homogenous_background[cell_roi_masks_union == 0] = background_color

  cell_rois_wo_neighbor = []
  cell_rois = []
  for i in range(len(cell_roi_masks)):
    mask = cell_roi_masks[i]
    if not mask.any(): 
        continue;
    
    cell_roi = np.copy(img_homogenous_background)
    cell_roi[mask == 0] = 0
    cell_rois.append(crop_image_only_outside(cell_roi))

    cell_roi_wo_neighbor = np.copy(img_homogenous_background)

    for j in range(rescaled_masks.shape[2]):

      if i != j:
        cell_roi_wo_neighbor[rescaled_masks[:,:,j].astype('int') > 0] = background_color
      
    cell_roi_wo_neighbor[mask == 0] = 0
    cell_rois_wo_neighbor.append(crop_image_only_outside(cell_roi_wo_neighbor))
  return (cell_rois, cell_rois_wo_neighbor)     


In [0]:
def handle_dir(image_dir, target_magnification=10):
  """Full pipeline to segment all images inside target directory into individual cells"""

  print(image_dir)
  # extract magnification from directory name
  magnification = int(os.path.basename(image_dir))

  images = glob.glob(os.path.join(image_dir,'*'))
  print(images)

  """ The pretrained Mask RCNN model seens to works best with tiles of size 320x320px and a magnification of 10 (tested empirically).
  So we need to rescale all images of other magnifications and split them into tiles.
  E.g. to transform an image of size 2560x1920px with magnification of 60x into a tile of size 320x320px with magnification 10x,
  we would need to extract a tile of size 1920x1920px from the original image and downscale it with a factor of 10/60.
  """
  target_tile_size = int(32 * target_magnification) 
  scale_factor = target_magnification / magnification
  original_tile_size = int(target_tile_size / scale_factor)

  for img_path in images:
    base_filename = Path(img_path).stem
    img = skimage.io.imread(img_path) #
    print(img_path)
    # First split the original image into the required tiles. E.g. 1920x1920px tile from above.
    original_tiles = np.array(img_to_tiles(img, original_tile_size))

    for i in range(len(original_tiles)):
      filename = base_filename+'_tile'+str(i)
      original_tile = original_tiles[i]  

      # Here we rescale our image to target size and do some more image preparation to improve segmentation
      augmented_tile = prepare_image(original_tile, scale_factor)
      # The model can be used on multiple images at once, but single processing makes saving the results and debugging easier 
      results = nmodel.detect([augmented_tile], verbose=1)
      r = results[0]
      """The Mask RCNN model returns two results for each segmented cell, which are useful for our purpose:
      1) Bounding boxes around the segmented cells called region of interest (r['rois']) in the shape [xmin, ymin, xmax, ymax]. 
         r['rois'].shape == (nr_of_cells, 4)
      2) Binary masks (r['masks']) in the shape of the rescaled image. The last dimension of the nparray is for the segmented cells
       =>r['masks'].shape == (320, 320, nr_of_cells), with target_tile_size = 320 
      """
      
      # Because we want to apply the segmentation results to our original image, 
      # we need to scale back the results from our model 
      rescaled_rois = r['rois'] * 1/scale_factor
      rescaled_masks = skimage.transform.rescale(r['masks'], (1/scale_factor, 1/scale_factor, 1))

      _, ax = plt.subplots(1, figsize=(16,16))
      # Helper function from original Matterport repository to display Mask RCNN results
      # Visualize (and save) colored cell outline/boundaries inside original tile  
      visualize.display_instances(original_tile, rescaled_rois, rescaled_masks,r['class_ids'],['BG', 'Nuclei'],
                      ax=ax, show_mask=False, show_bbox=False,
                      title="Predictions")  
      save_figure_results("tile_cells_outline", magnification, filename)

      # Visualize (and save) colored cell overlay/mask inside original tile  
      _, ax = plt.subplots(1, figsize=(16,16))
      visualize.display_instances(original_tile, rescaled_rois, rescaled_masks,r['class_ids'],['BG', 'Nuclei'],
                      ax=ax, show_mask=True, show_bbox=False,
                      title="Predictions") 
      save_figure_results("tile_cells_masked_dir", magnification, filename)
       
      # Now we move away from visualizing the complete tile and instead display individual cells 
      # First we transform the roi results into individual masks, which are still the size of the original tile   
      roi_masks = np.array(rois_to_masks(rescaled_rois, original_tile))
      roi_masks = np.moveaxis(roi_masks, 0, -1) # ((roi_masks.shape[2], roi_masks.shape[1], roi_masks.shape[0]))
      
      # Next we apply the individual cell masks to the original tile and save only the resulting segmented cells.
      # Here we save the individual cells as rectangular extraction the size of their roi from the original tile
      save_cell_masks(filename, magnification, 'cell_rois', original_tile, roi_masks)
      # Here we save the individual cells as extraction still the size of their roi from the orignal tile,
      # but now every pixel not segmented as cell is set to (0,0,0)
      save_cell_masks(filename, magnification, 'cell_masks', original_tile, rescaled_masks)

      # Because the model is pretrained on nuclei, the results usually do not contain the whole cell. 
      # Therefore the next step is extending the roi and repeating the above extraction from the original tile. 
      extended_rois = get_extended_cell_rois(r, original_tile, rescaled_rois)
      rescaled_cell_roi_masks = rois_to_masks(extended_rois, original_tile)
      
      # Visualize (and save) rectangular extraction the size of their extended roi from the original tile
      _, ax = plt.subplots(1, figsize=(16,16))
      visualize.display_instances(original_tile, extended_rois, np.zeros(r['masks'].shape),r['class_ids'],['BG', 'Nuclei'],
                      ax=ax, show_mask=False, show_bbox=True,
                      title="Predictions")  
      save_figure_results("tile_cells_extended_boxes", magnification, filename)
      
      # When we extend the roi of each cell, we naturally start to create a stronger overlap between cells.
      # To prevent multiple cells in the same picture, for every cell we set all pixels belonging to other cells to 
      # the average background color of all non-cell pixels (cell_rois_extended_wo_neighbors)
      rescaled_cell_roi_masks_union = unify_cell_roi_masks(original_tile, rescaled_cell_roi_masks)
      background_color = get_avg_background_color(original_tile, rescaled_cell_roi_masks_union)
      cell_rois_extended, cell_rois_extended_wo_neighbors = get_segmented_cells_rois(original_tile, rescaled_masks, rescaled_cell_roi_masks, rescaled_cell_roi_masks_union, background_color)
      # Here we save the individual cells as rectangular extraction the size of their extended roi from the orignal tile
      save_segmented_cells(filename, magnification, 'cell_rois_extended', cell_rois_extended)
      # Here we save the individual cells the same as "cell_rois_extended", but with neighbor cells replaced by 
      # the average background color of all non-cell pixels (cell_rois_extended_wo_neighbors)
      save_segmented_cells(filename, magnification, 'cell_rois_extended_wo_neighbors', cell_rois_extended_wo_neighbors)
 

In [0]:
image_dirs=glob.glob(os.path.join(DATASET_DIR, "*"))
image_dirs

In [0]:
# Takes VERY long. You should try individual directories first. E.g. [image_dirs[0]]
for image_dir in image_dirs: 
  target_magnification = 10
  handle_dir(image_dir, target_magnification)